# Overfitting and Regularization

## What is overfitting?

- Overfitting is when your model is too complicated and can't generalize to unseen data well
- If you are overfitting in linear regression, this could meanyou have too many features/inputs in your model (too many x variables)
- You're fitting to noise rather than the underlying data trend
- You can think of overfitting as memorizing data rather than learning the trend

## Linear Regression with many inputs/independent variables

- The data below has 100 people's salaries (in thousands of dollars), years of work experience, years of education, number of pizza slices eaten in a year, and 100 other random info about them.

In [ ]:
import torch
import plotly.express as px
import plotly.graph_objects as go
from copy import deepcopy
import pandas as pd
torch.manual_seed(0)

In [ ]:
df = torch.load('multivar_lin_reg.pt')
df

- The true output or $\bm{y}$ is the `salaries` column

- Let's assume you don't know which of the other columns are correlated with `salaries`, and you decide to run a linear regression with all 103 columns as the $x$ variables.

- You'll use mean square error as your objective/loss function

- You'll try to learn the $b$ and $m$'s (intercept/bias and slopes/coefficents) that minimizes/optimizes our loss/objective function:
  $$\underset{b, m_1, m_2, \dots, m_{103}}{\operatorname{argmin}} \  \frac{1}{N} \sum_{n=1}^{N} { \left( \hat{y}^{(n)} - y^{(n)} \right) ^2} $$
  - where:
  $$ \hat{y}^{(n)} = b + m_1 x_1^{(n)} + m_2 x_2^{(n)} + ... + m_{103} x_{103}^{(n)} $$

- or in matrix form:

  $$ \underset{\bm{w}}{\operatorname{argmin}} \  \frac{1}{N} (\hat{\bm{y}}-\bm{y})^\top (\hat{\bm{y}}-\bm{y})  $$

    - where
    
    $$ \hat{\bm{y}} = \bm{X}\bm{w} $$

    $$ \bm{X} = \left[\begin{array}{cc}
    1 & x_1^{(1)} & x_2^{(1)} & \dots & x_{103}^{(1)} \\
    1 & x_1^{(2)} & x_2^{(2)} & \dots & x_{103}^{(2)} \\
    \vdots & \vdots & \vdots & \ddots  & \vdots\\
    1 & x_1^{(100)} & x_2^{(100)} & \dots & x_{103}^{(100)}
   \end{array}\right], \quad \quad 
    \bm{w} = \left[\begin{array}{cc}
    b \\
    m_1 \\
    \vdots \\
    m_{103}
   \end{array}\right] $$

## Standardization of data

### Steps to standardize

- take the mean, $\mu$ and standard deviation, $\sigma$, of each column
- subtract every value by its column's mean
- then divide each value by its column's standard deviation


### Why standardize data?

- to get every variable on the same scale
  - our `salaries` column goes from about 0-400, `exp_years` 0-40, `ed_years` 9-22, 
  - every column will have $\mu=0$ and $\sigma=1$ after standardization
- makes gradient descent behave numerically better
  - less likely to lose precision from very large or very small gradients
- necessary with **regularization** (will be discussed later)


In [ ]:
def standardize(data):
    assert isinstance(data, torch.Tensor) # check that data is a pytorch tensor
    column_mu = data.mean(dim=0) # take the mean of each column
    column_sigma = data.std(dim=0) # take the std deviation of each column
    data_standardized = (data-column_mu)/column_sigma # subract each values column mean then divide by the column std deviation
    return data_standardized, column_mu, column_sigma

def unstandardize(data_stand, column_mu, column_sigma):
    assert isinstance(data_stand, torch.Tensor)
    return data_stand * column_sigma + column_mu

In [ ]:
data_stand, mu, sig = standardize(torch.tensor(df.values))
df_stand = pd.DataFrame(data_stand).astype(float)
df_stand.columns = df.columns
df_stand

## Split data into train, validation, and test sets

- The train data is what we use to do the optimization/minimization of the objective/loss function
    - Last lesson everything was training data
- The validation data is what we use to check if we're overfitting during training
- The test data is what we use to report how well our model does
    - You should only use test data once
    - If you don't like the results on the test data:
        - turn the test data into training or validation data and re-train the model
        - get new data that you've never seen before to be the new test data
- If you are overfitting, you will see your loss function increasing with the validation data while decreasing with the training data

In [ ]:
X_all = torch.tensor(df_stand.iloc[:,1:].values)
Y_all = torch.tensor(df_stand['salaries'])

In [ ]:
def mse(pred_Y, true_Y):
    assert pred_Y.shape == true_Y.shape
    return torch.square(pred_Y - true_Y).mean()

In [ ]:
random_order_indices = torch.randperm(len(X_all)) # random order of X_all's indices
train_proportion = 0.7
valid_proportion = 0.15
test_proportion = 0.15
train_valid_index_border = torch.math.floor(len(X_all)*train_proportion)
valid_test_index_border = torch.math.floor(len(X_all)*(train_proportion+valid_proportion))
train_indices = random_order_indices[0:train_valid_index_border] # take first 80% of random_order_indices to be the train indices
valid_indices = random_order_indices[train_valid_index_border:valid_test_index_border]
test_indices = random_order_indices[valid_test_index_border:]

X = X_all[train_indices]
Y = Y_all[train_indices]
X_valid = X_all[valid_indices]
Y_valid = Y_all[valid_indices]
X_test  = X_all[test_indices]
Y_test  = Y_all[test_indices]

Make sure distribution of each set is similar

In [ ]:
fig = go.Figure()
fig.update_layout( height=500, width=750, margin=dict(l=20, r=20, t=20, b=20) )
train_trace = go.Histogram( x=Y, histnorm='percent', name='train' )
valid_trace = go.Histogram( x=Y_valid, histnorm='percent', name='validation')
test_trace = go.Histogram( x=Y_test, histnorm='percent', name='test')
fig.add_traces([train_trace, valid_trace, test_trace])

## Overfit to the random data columns in the data

### Using Pytorch's linear module

- This time we'll use pytorch's Linear module and pytorch's gradient descent optimizer

- The Linear module stores the $b$ and $m$'s separately in `bias` and `weight`properties of the module

- make a prediciton by calling the module with input data as the paramters

In [ ]:
line = torch.nn.Linear(in_features=103, out_features=1) # in_features is the number of x-variables, out_features is the number of outputs per data entry
line.weight # contains the coefficients of the inputs/x variables
line.bias # contains the bias/y-intercept
line(X[0,:].float()) # make a predicion with the current weight and bias using the first train data entry as input

### Using Pytorch's gradient descent optimizer

- we're going to use pytorch's Stochastic Gradient Descent method to do gradient descent

    - *Stochastic* part of this will be explained in a later lesson
    
    - we're using this as just a regular gradient descent here

In [ ]:
optim = torch.optim.SGD(line.parameters(), lr=0.05) # use gradient descent with learning rate of 0.05

- Make a function to do a step of gradient descent

In [ ]:
def step(X, Y, X_valid, Y_valid, line, optim, Losses, Losses_valid):
    global epoch
    print('epoch #{}'.format(epoch))
    optim.zero_grad() # reset the gradients
    line.train() # tells pytorch we're training the linear module

    pred_y = line(X.float()).squeeze() # make a prediction
    # squeeze removes dimensions with size 1
    #   e.g. squeeze applied to a tensor with shape (3,1) will return a tensor with shape (3)

    loss = mse(pred_y, Y) # calculate mse
    loss.backward() # calculate gradients
    optim.step() # take gradient descent step

    line.eval() # tells pytorch we're not training our linear module (we're validating now instead of training)
    with torch.no_grad(): # Tells pytorch to not track gradients. This prevents unnecessary calculations and memory usage
        pred_y_valid = line(X_valid.float()).squeeze()
        loss_valid = mse(pred_y_valid, Y_valid)

    Losses.append(loss.item()) # add current training loss to Losses list
    Losses_valid.append(loss_valid.item()) # add current validation loss to Losses_valid list
    print('Validation Loss: ' + str(loss_valid.item()))
    epoch += 1

- Make a function to plot the MSE v gradient descent steps

In [ ]:
def graph_steps(title):
    Losses = []
    Losses_valid = []
    fig = go.FigureWidget()
    trace_train = go.Scatter( x=list(range(len(Losses))), y=Losses, line=dict(color=px.colors.qualitative.Plotly[0]), name='train' )
    fig.add_trace(trace_train)
    trace_valid = go.Scatter( x=list(range(len(Losses_valid))), y=Losses_valid, line=dict(color=px.colors.qualitative.Plotly[1]), name='valid' )
    fig.add_trace(trace_valid)
    # color=px.colors.qualitative.Plotly[1] 
    # fig.add_traces( list( px.line( x=range(len(Losses_valid)), y=Losses_valid, color=px.colors.qualitative.Plotly[1]  ).select_traces() ) )
    fig.update_layout( autosize=False, width=750, height=500, title=title )
    fig.update_xaxes( title="steps" )
    fig.update_yaxes( title='MSE')
    return fig

- Create function to copy linear models. Use this to store your best models.

In [ ]:
def copy_linear(linear):
    in_features = linear.weight.shape[1]
    out_features = linear.weight.shape[0]
    linear_copy = torch.nn.Linear(in_features, out_features)
    linear_copy.load_state_dict(linear.state_dict())
    return linear_copy

### Train model until it overfits

In [ ]:
Losses = []
Losses_valid = []
epoch = 0
best_line = None # best_line holds the best model learned
best_line_valid_loss = torch.inf # holds the loss value on validation data from the best_line
fig = graph_steps('Linear regression gradient descent')
train_loss = fig.data[0] # use this variable to add train losses to graph
valid_loss = fig.data[1] # use this variable to add validation losses to graph
fig # display the graph

In [ ]:
step(X, Y, X_valid, Y_valid, line, optim, Losses, Losses_valid) # take a gradient descent step
if Losses_valid[-1] < best_line_valid_loss:
    best_line = copy_linear(line)
    best_line_valid_loss = Losses_valid[-1]
train_loss.x = list(range(len(Losses))) # add the train steps to graph on x-axis
train_loss.y = Losses # add the train losses to graph on y-axis
valid_loss.x = list(range(len(Losses_valid)))
valid_loss.y = Losses_valid

Keep running the code block above until the MSE for the validation data is increasing while the training data MSE is decreasing.

This means you are overfitting!

## Regularization

- Regularization are methods used to prevent overfitting
- There are two main regularization techniques for linear regression: L1/Lasso and L2/Ridge

### L1 Lasso

- steps:

    - Take the absolute value of all the weights/slopes

    
    - Scale it by mulitplying by a regularization factor, $\lambda$, that you choose
    
    - Add it to your MSE

- Your regularized loss/objective function will then be:
    
    $$\underset{b, m_1, m_2, \dots, m_{I}}{\operatorname{argmin}} \  \frac{1}{N} \sum_{n=1}^{N} { \left( \hat{y}^{(n)} - y^{(n)} \right) ^2} + \lambda \sum_{i=1}^{I}{ \left| m_i \right| } $$
    
    - where:
    
        - $N$ is the number of data samples and
    
        - $I$ is the number of independent/x-variables

### L2 Ridge

- similar to L1 Lasso regularization, but square the values instead of taking the absolute value

- Your regularized loss/objective function with L2/Ridge regularization will be:

    $$\underset{b, m_1, m_2, \dots, m_{I}}{\operatorname{argmin}} \  \frac{1}{N} \sum_{n=1}^{N} { \left( \hat{y}^{(n)} - y^{(n)} \right) ^2} + \lambda \sum_{i=1}^{I}{m_i^2} $$

### When to use L1 vs L2

- $\lambda$ controls the strength of regularization for both:
- $\lambda=0$ is the same as no regularization
- L1/Lasso regression drives coefficients/weights/slopes to zero
  - The higher the $\lambda$ the more coefficients will become zero
- L2/Rigde regression is good to use when your x-variables are correlated

### Coding L1 and L2 loss functions

In [ ]:
def mse(pred_Y, true_Y):
    assert pred_Y.shape == true_Y.shape
    return torch.square(pred_Y - true_Y).mean()

def mse_l1(pred_Y, true_Y, linear, lam):
    assert pred_Y.shape == true_Y.shape
    reg = lam * linear.weight.abs().sum()
    return torch.square(pred_Y - true_Y).mean() + reg

def mse_l2(pred_Y, true_Y, linear, lam):
    assert pred_Y.shape == true_Y.shape
    reg = lam * linear.weight.square().sum()
    return torch.square(pred_Y - true_Y).mean() + reg

- change the step function to change the loss function based on the regularizer you want to use

In [ ]:
def step(X, Y, X_valid, Y_valid, lam, line, optim, regularizer, Losses, Losses_valid):
    global epoch
    print('epoch #{}'.format(epoch))
    optim.zero_grad() # reset the gradients
    line.train()

    pred_y = line(X.float()).squeeze() #

    if regularizer == None:
        loss = mse(pred_y, Y)
    elif regularizer == 'l1':
        loss = mse_l1(pred_y, Y, line, lam)
    elif regularizer == 'l2':
        loss = mse_l2(pred_y, Y, line, lam)
    loss.backward()
    optim.step()

    line.eval()
    with torch.no_grad():
        pred_y_valid = line(X_valid.float()).squeeze()
        loss_valid = mse(pred_y_valid, Y_valid)
    line.train()
    
    Losses.append(loss.item())
    Losses_valid.append(loss_valid.item())
    print('Validation Loss: ' + str(loss_valid.item()))
    epoch += 1

### Train L1 model

In [ ]:
Losses_l1 = []
Losses_valid_l1 = []
epoch = 0
lam = 0.25
line_l1 = torch.nn.Linear(in_features=X.shape[1], out_features=1)
best_line_l1 = None # best_line holds the best model learned
best_line_valid_loss_l1 = torch.inf # holds the loss value on validation data from the best_line
optim_l1 = torch.optim.SGD(line_l1.parameters(), lr=0.01) # use gradient descent  with learning rate of 0.1
regularizer = 'l1'
fig_l1 = graph_steps(regularizer)
train_loss_l1 = fig_l1.data[0]
valid_loss_l1 = fig_l1.data[1]
fig_l1

In [ ]:
step(X, Y, X_valid, Y_valid, lam, line_l1, optim_l1, regularizer, Losses_l1, Losses_valid_l1)
if Losses_valid_l1[-1] < best_line_valid_loss_l1:
    best_line_l1 = copy_linear(line_l1)
    best_line_valid_loss_l1 = Losses_valid_l1[-1]
train_loss_l1.x = list(range(len(Losses_l1)))
train_loss_l1.y = Losses_l1
valid_loss_l1.x = list(range(len(Losses_valid_l1)))
valid_loss_l1.y = Losses_valid_l1

### Train the L2 model

In [ ]:
Losses_l2 = []
Losses_valid_l2 = []
epoch = 0
lam = 0.25
line_l2 = torch.nn.Linear(in_features=X.shape[1], out_features=1)
best_line_l2 = None # best_line holds the best model learned
best_line_valid_loss_l2 = torch.inf # holds the loss value on validation data from the best_line
optim_l2 = torch.optim.SGD(line_l2.parameters(), lr=0.05) # use gradient descent  with learning rate of 0.1
regularizer = 'l2'
fig_l2 = graph_steps(regularizer)
train_loss_l2 = fig_l2.data[0]
valid_loss_l2 = fig_l2.data[1]
fig_l2

In [ ]:
step(X, Y, X_valid, Y_valid, lam, line_l2, optim_l2, regularizer, Losses_l2, Losses_valid_l2)
if Losses_valid_l2[-1] < best_line_valid_loss_l2:
    best_line_l2 = copy_linear(line_l2)
    best_line_valid_loss_l2 = Losses_valid_l2[-1]
train_loss_l2.x = list(range(len(Losses_l2)))
train_loss_l2.y = Losses_l2
valid_loss_l2.x = list(range(len(Losses_valid_l2)))
valid_loss_l2.y = Losses_valid_l2

### Compare to ground-truth

- The only two columns/variables correlated with salaries is education and experience

- I only know this because I created the data. It's not real world data.

- In a real dataset you will not know the actual true trend. (If you already knew it, you wouldn't need to do a linear regression!)

- The underlying line (unstandardized) I used to create this data is:
  $$y = -61.3846 + 9.8462 x_1 + 7.6154 x_2$$
  - $y$ is `salaries`
  - $x_1$ is `exp_years`
  - $x_2$ is `ed_years`

In [ ]:
mu_salaries = mu[0]
mu_exp = mu[1]
mu_ed = mu[2]
sig_salaries = sig[0]
sig_exp = sig[1]
sig_ed = sig[2]

bias_std = (-61.3846 + 9.8462*mu_exp + 7.6154*mu_ed - mu_salaries)/sig_salaries
weight_std_1 = 9.8462*sig_exp/sig_salaries
weight_std_2 = 7.6154*sig_ed/sig_salaries
print("y = {} + {} x1 + {} x2".format(bias_std, weight_std_1, weight_std_2))

- The standardized version of the underlying line is:
  $$y = 0.05399 + 0.82508 x_1 + 0.20634 x_2$$

In [ ]:
Losses_2x = []
Losses_valid_2x = []
epoch = 0
line_2x = torch.nn.Linear(in_features=2, out_features=1)
line_2x.weight.data = torch.tensor([[0.82508, 0.20634]])
line_2x.bias.data = torch.tensor([0.05399])
fig_2x = graph_steps('Linear regression gradient descent')
train_loss_2x = fig.data[0] # use this variable to add train losses to graph
valid_loss_2x = fig.data[1] # use this variable to add validation losses to graph

valid_2x_pred_y = line_2x(X_valid[:,0:2].float()).squeeze()
valid_2x_mse = mse(valid_2x_pred_y, Y_valid)

valid_2x_mse.item()

#### Graph the validation losses

In [ ]:
fig = go.Figure()
trace_valid_no_reg = go.Scatter( x=torch.tensor(range(len(Losses_valid)))/len(Losses_valid), y=Losses_valid, name='valid_no_reg' )
trace_valid_L1 = go.Scatter( x=torch.tensor(range(len(Losses_valid_l1)))/len(Losses_valid_l1), y=Losses_valid_l1, name='valid_L1' )
trace_valid_l2 = go.Scatter( x=torch.tensor(range(len(Losses_valid_l2)))/len(Losses_valid_l2), y=Losses_valid_l2, name='valid_L2' )
trace_2x = go.Scatter ( x=[0,1], y=[valid_2x_mse.detach().clone(), valid_2x_mse.detach().clone()], name='valid_underlying_line' )
fig.add_traces([trace_valid_no_reg, trace_valid_L1, trace_valid_l2, trace_2x])
fig.update_xaxes(title="Training Steps [proportion]")
fig.update_yaxes(title="MSE")
fig

#### Graph the coefficients/slopes

In [ ]:
fig = go.Figure( data= [
    go.Bar(name='no reg', x=df.columns[1:], y=best_line.weight.data.squeeze().tolist()),
    go.Bar(name='L2', x=df.columns[1:], y=best_line_l2.weight.data.squeeze().tolist()),
    go.Bar(name='L1', x=df.columns[1:], y=best_line_l1.weight.data.squeeze().tolist()),
    go.Bar(name='underlying', x=df.columns[1:3], y=line_2x.weight.data.squeeze().tolist()) ])
fig.update_layout(barmode='group')
fig

### Evaluate different models on test data

- We are going to calculate the MSE with unstandardized values on test data
- We are then going to take the square root of the MSE
    - This is called the Root Mean Square Error (RMSE)
    - this takes it back into the same scale as our original salaries

#### RMSE Unregularized

In [ ]:
test_pred_y = best_line(X_test.float()).squeeze()
pred_y = unstandardize(test_pred_y, mu[0], sig[0])
test_y_unstand = unstandardize(Y_test, mu[0], sig[0])
test_no_reg_mse = mse(pred_y, test_y_unstand)
test_no_reg_mse.sqrt().item()

#### RMSE L1/Lasso

In [ ]:
test_pred_y = best_line_l1(X_test.float()).squeeze()
pred_y = unstandardize(test_pred_y, mu[0], sig[0])
test_y_unstand = unstandardize(Y_test, mu[0], sig[0])
test_L1_mse = mse(pred_y, test_y_unstand)
test_L1_mse.sqrt().item()

#### RMSE L2/Ridge

In [ ]:
test_pred_y = best_line_l2(X_test.float()).squeeze()
pred_y = unstandardize(test_pred_y, mu[0], sig[0])
test_y_unstand = unstandardize(Y_test, mu[0], sig[0])
test_L2_mse = mse(pred_y, test_y_unstand)
test_L2_mse.sqrt().item()

#### RMSE Underlying Line

In [ ]:
test_pred_y = line_2x(X_test[:,0:2].float()).squeeze()
pred_y = unstandardize(test_pred_y, mu[0], sig[0])
test_y_unstand = unstandardize(Y_test, mu[0], sig[0])
test_2x_mse = mse(pred_y, test_y_unstand)
test_2x_mse.sqrt().item()